In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def census(target,lis):
    answer=0
    for n in lis:
        if n==target:answer=answer+1
    return answer

def positions(target,lst):
    ln=len(lst)
    answer=[]
    for j in [0..ln-1]:
        entry=lst[j]
        if entry==target:answer=answer+[j+1]
    return(answer)

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    


def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [7]:
# verify narrow version, p = 3
import time

def p3RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        d=order(3,a)
        purepart=3^d
        oddpart=a/3^d
        term=4^d*oddpart^2*sigma(oddpart,k=7)
        ans=ans+term*x^a
    return D(ans)

def recipP3(n):
    ans=L(1/p3RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p3RecipPowerConstantTerm(k):
    g=recipP3(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

ls=[]
start=time.time()
for a in [1..20]: #default precision must be > max a.
    od=order(3,a)
    b=ZZ(a/3^od)
    c=p3RecipPowerConstantTerm(b)
    o3=order(3,c)
    ds=digitsum(3,a)
    test=ds-o3
    finish=time.time()
    print((a,ds,o3,finish-start))
    if test!=0:ls=ls+[a]
print()
print("no-count:");print();print(len(ls))
print("bad a's:");print();print(ls)

(1, 1, 1, 0.01751232147216797)
(2, 2, 0, 0.0408482551574707)
(3, 1, 1, 0.056351423263549805)
(4, 2, 0, 0.08791446685791016)
(5, 3, 1, 0.12341523170471191)
(6, 2, 0, 0.14391827583312988)
(7, 3, 1, 0.2153153419494629)
(8, 4, 0, 0.2781252861022949)
(9, 1, 1, 0.2929401397705078)
(10, 2, 0, 0.3522224426269531)
(11, 3, 1, 0.4140584468841553)
(12, 2, 0, 0.4437544345855713)
(13, 3, 1, 0.5038652420043945)
(14, 4, 0, 0.5630221366882324)
(15, 3, 1, 0.5973801612854004)
(16, 4, 0, 0.6557183265686035)
(17, 5, 1, 0.7137594223022461)
(18, 2, 0, 0.7353992462158203)
(19, 3, 1, 0.8015282154083252)
(20, 4, 0, 0.8656423091888428)

no-count:

17
bad a's:

[2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [8]:
# verify narrow version, p = 3
import time

def p3RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        d=order(3,a)
        purepart=3^d
        oddpart=a/3^d 
        term=9^d*oddpart^2*sigma(oddpart,k=7)
        # here I pretend that ord(3,3^a)= 3a; this seems to work better
        ans=ans+term*x^a
    return D(ans)

def recipP3(n):
    ans=L(1/p3RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p3RecipPowerConstantTerm(k):
    g=recipP3(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

ls=[]
start=time.time()
for a in [1..20]: #default precision must be > max a.
    od=order(3,a)
    b=ZZ(a/3^od)
    c=p3RecipPowerConstantTerm(b)
    o3=order(3,c)
    ds=digitsum(3,a)
    test=ds-o3
    finish=time.time()
    print((a,ds,o3,finish-start))
    if test!=0:ls=ls+[a]
print()
print("no-count:");print();print(len(ls))
print("bad a's:");print();print(ls)

(1, 1, 1, 0.016028881072998047)
(2, 2, 2, 0.036804914474487305)
(3, 1, 1, 0.05214095115661621)
(4, 2, 2, 0.0839548110961914)
(5, 3, 3, 0.11929702758789062)
(6, 2, 2, 0.13969779014587402)
(7, 3, 3, 0.20120000839233398)
(8, 4, 4, 0.2603130340576172)
(9, 1, 1, 0.2758629322052002)
(10, 2, 2, 0.3363058567047119)
(11, 3, 3, 0.3954339027404785)
(12, 2, 2, 0.4250800609588623)
(13, 3, 3, 0.4836430549621582)
(14, 4, 4, 0.5416438579559326)
(15, 3, 3, 0.5745389461517334)
(16, 4, 4, 0.6319637298583984)
(17, 5, 5, 0.6950869560241699)
(18, 2, 2, 0.7143218517303467)
(19, 3, 3, 0.7741930484771729)
(20, 4, 4, 0.8366827964782715)

no-count:

0
bad a's:

[]


In [46]:
#v1 is the list `od' from cell 37
v1=vector([0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 5, 2, 2, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 5, 2, 1, 1, 1, 3, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 2, 4, 1, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 1, 2, 4, 2, 1, 1, 2, 2, 3, 3, 2, 3, 3, 3, 4, 2, 3, 3, 3, 1, 1, 2, 2, 2, 1, 1, 2, 3, 2, 1, 1, 2, 2, 4, 2, 3, 4, 3, 3, 2, 4, 3, 3, 3, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 2, 2, 2, 1, 1, 3, 2, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 3, 2, 1, 1, 1, 5, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 4, 1])
v2=vector(od)
print(v1-v2)

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [6]:
# verify narrow version, p = 7
import time

def p7RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=3)
        ans=ans+term*x^a
    return D(ans)

def recipP7(n):
    ans=L(1/p7RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p7RecipPowerConstantTerm(k):
    g=recipP7(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

one=[]
zero=[]
od=[]
no=[]
start=time.time()
for a in [1..200]: #default precision must be > max a.
    c=p7RecipPowerConstantTerm(a)
    o7=order(7,c)
    od=od+[o7]
    finish=time.time()
    print((a,o7,finish-start))
print();print("od:");print(od)

(1, 0, 0.013863325119018555)
(2, 0, 0.021677017211914062)
(3, 0, 0.031173229217529297)
(4, 0, 0.042696237564086914)
(5, 1, 0.05498814582824707)
(6, 1, 0.07217097282409668)
(7, 0, 0.09158825874328613)
(8, 0, 0.11175823211669922)
(9, 0, 0.1314101219177246)
(10, 0, 0.15090417861938477)
(11, 0, 0.17081522941589355)
(12, 1, 0.19075298309326172)
(13, 1, 0.21055936813354492)
(14, 0, 0.23012828826904297)
(15, 0, 0.24996018409729004)
(16, 0, 0.2697451114654541)
(17, 0, 0.28951597213745117)
(18, 0, 0.3087780475616455)
(19, 3, 0.3282182216644287)
(20, 1, 0.35417604446411133)
(21, 0, 0.3772270679473877)
(22, 0, 0.39913129806518555)
(23, 0, 0.42086338996887207)
(24, 0, 0.4427921772003174)
(25, 0, 0.4645202159881592)
(26, 1, 0.48651814460754395)
(27, 1, 0.5084691047668457)
(28, 0, 0.5306642055511475)
(29, 0, 0.5530450344085693)
(30, 0, 0.5755870342254639)
(31, 0, 0.5975983142852783)
(32, 0, 0.6194121837615967)
(33, 1, 0.6420800685882568)
(34, 2, 0.6643900871276855)
(35, 1, 0.6866621971130371)
(36, 2

In [21]:
no=[]
for n in[1..200]:
    c5=census(5,n.digits(7))
    c6=census(6,n.digits(7))
    if c5==0:
        if c6==0:
            no=no+[n]
print(no);print()
print(ps);print()
print((len(no),len(ps)))
vno=vector(no)
vps=vector(ps)
print();print(vno-vps)

[1, 2, 3, 4, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 28, 29, 30, 31, 32, 49, 50, 51, 52, 53, 56, 57, 58, 59, 60, 63, 64, 65, 66, 67, 70, 71, 72, 73, 74, 77, 78, 79, 80, 81, 98, 99, 100, 101, 102, 105, 106, 107, 108, 109, 112, 113, 114, 115, 116, 119, 120, 121, 122, 123, 126, 127, 128, 129, 130, 147, 148, 149, 150, 151, 154, 155, 156, 157, 158, 161, 162, 163, 164, 165, 168, 169, 170, 171, 172, 175, 176, 177, 178, 179, 196, 197, 198, 199, 200]

[1, 2, 3, 4, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 28, 29, 30, 31, 32, 49, 50, 51, 52, 53, 56, 57, 58, 59, 60, 63, 64, 65, 66, 67, 70, 71, 72, 73, 74, 77, 78, 79, 80, 81, 98, 99, 100, 101, 102, 105, 106, 107, 108, 109, 112, 113, 114, 115, 116, 119, 120, 121, 122, 123, 126, 127, 128, 129, 130, 147, 148, 149, 150, 151, 154, 155, 156, 157, 158, 161, 162, 163, 164, 165, 168, 169, 170, 171, 172, 175, 176, 177, 178, 179, 196, 197, 198, 199, 200]

(104, 104)

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
# verify narrow version, p = 7
import time

def p7RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=rmnjntau(a)
        ans=ans+term*x^a
    return D(ans)

def recipP7(n):
    ans=L(1/p7RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p7RecipPowerConstantTerm(k):
    g=recipP7(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

one=[]
zero=[]
od=[]
no=[]
start=time.time()
for a in [1..200]: #default precision must be > max a.
    c=p7RecipPowerConstantTerm(a)
    o7=order(7,c)
    od=od+[o7]
    finish=time.time()
    print((a,o7,finish-start))
print();print("od:");print(od)

(1, 0, 0.009374380111694336)
(2, 0, 0.019326210021972656)
(3, 0, 0.03057098388671875)
(4, 0, 0.04056715965270996)
(5, 1, 0.05084729194641113)
(6, 1, 0.06532502174377441)
(7, 0, 0.08088231086730957)
(8, 0, 0.09607720375061035)
(9, 0, 0.11124134063720703)
(10, 0, 0.1255490779876709)
(11, 0, 0.13991808891296387)
(12, 2, 0.15432119369506836)
(13, 1, 0.16839814186096191)
(14, 0, 0.18260407447814941)
(15, 0, 0.19708800315856934)
(16, 0, 0.21193432807922363)
(17, 0, 0.22670722007751465)
(18, 0, 0.24076008796691895)
(19, 1, 0.2550961971282959)
(20, 2, 0.26985597610473633)
(21, 0, 0.28543996810913086)
(22, 0, 0.30115318298339844)
(23, 0, 0.3171210289001465)
(24, 0, 0.3340911865234375)
(25, 0, 0.35120105743408203)
(26, 1, 0.36891627311706543)
(27, 1, 0.3874831199645996)
(28, 0, 0.4060642719268799)
(29, 0, 0.4251241683959961)
(30, 0, 0.4444611072540283)
(31, 0, 0.4644291400909424)
(32, 0, 0.48500919342041016)
(33, 1, 0.505120038986206)
(34, 1, 0.5260763168334961)
(35, 1, 0.5474631786346436)
(36, 